# Binding prediction using AptaNet
Step-by-step guide for using AptaNet for binary aptamer–protein binding prediction.

## Overview
This notebook showcases the AptaNet algorithm, a deep learning method that combines sequence-derived (k-mer + PSeAAC) features with RandomForest-based feature selection, and a multi-layer perceptron to predict whether an aptamer and a protein interact (binary classification: aptamer binds/does not bind with the target protein). An overview of the classes and helper functions used in this notebook:

- **pairs_to_features**: helper that converts `(aptamer_sequence, protein_sequence)` pairs into feature vectors using k-mer + PSeAAC.
- **SkorchAptaNet**: a PyTorch MLP wrapped in Skorch for binary classification.
- **load_1gnh_structure**: helper to load the 1GNH molecule structure from PDB file.

## Data preparation
To train the `AptaNetMLP` the notebook uses:
* For `X`:
    * 5 random aptamer sequences of length>30 (to satisfy the default lambda value of 30 set in the PSeAAC algorithm).
    * Amino acid sequences extracted from the 1GNH protein molecule.
    
    The aptamer sequences and the amino acid sequences form tuples `(aptamer_sequence, protein_sequence)` to form `X`.
* For `y`:
    * A random binary value (to indicate if the aptamer binds or not) as dummy data.

In [30]:
# Data imports
import torch

from pyaptamer.datasets import load_1gnh_structure
from pyaptamer.utils import struct_to_aaseq

In [31]:
aptamer_sequence = [
    "GGGAGGACGAAGACGACUCGAGACAGGCUAGGGAGGGA",
    "AAGCGUCGGAUCUACACGUGCGAUAGCUCAGUACGCGGU",
    "CGGUAUCGAGUACAGGAGUCCGACGGAUAGUCCGGAGC",
    "UAGCUAGCGAACUAGGCGUAGCUUCGAGUAGCUACGGAA",
    "GCUAGGACGAUCGCACGUGACCGUCAGUAGCGUAGGAGA",
]

gnh = load_1gnh_structure()
protein_sequence = struct_to_aaseq(gnh)

# Build all combinations (aptamer, protein)
X = [(a, p) for a in aptamer_sequence for p in protein_sequence]

# Dummy binary labels for the pairs
y = torch.randint(0, 2, (len(X),), dtype=torch.float32)

## Building the pipeline
### Dataset balancing using the Neighbourhood cleaning rule
In the `AptaNet` paper, the authors mention using the `NeighbourhoodCleaningRule` from `imblearn` in order to balance the dataset, as in their dataset they had more negative (0) values than positives (1).

 To build a scikit-learn pipeline, follow these steps:
1. Convert the input to the desired (aptamer_sequence, protein_sequence) format.
    * OPTIONAL: As mentioned in the paper, perform under-sampling using the  
    Neighborhood Cleaning Rule to balance the classes.
2. Get the PSeAAC feature vectors for your converted input (using `pairs_to_features`).
3. Select the number of features to use from the feature vector (using `RandomForestClassifier`).
4. Define the skorch neural network (using `AptaNetMLP`).
### Different workflows
In this first half of the notebook we will cover 3 different workflows one can follow, in ascending order of customizability:

1. A minimal workflow with no dataset balancing, while using the in-built pipeline.
2. Using your own custom pipeline.
3. Dataset balancing, while using your own pipeline.

### First workflow
A minimal workflow with no dataset balancing, while using the in-built pipeline.

In [32]:
# If you want to use the AptaNet pipeline, you can import it directly
from pyaptamer.aptanet import AptaNetPipeline

In [33]:
pipeline = AptaNetPipeline()

In [34]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

In [35]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.48


### Second Worflow

Your own custom-built pipeline.

In [36]:
# If you want to build your own aptamer pipeline, you should use the following imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from skorch import NeuralNetBinaryClassifier

from pyaptamer.aptanet._aptanet_nn import AptaNetMLP
from pyaptamer.utils._aptanet_utils import pairs_to_features

In [37]:
feature_transformer = FunctionTransformer(
    func=pairs_to_features,
    validate=False,
    # Optional arguments for pairs_to_features
    # example: kw_args={'k': 4, 'pseaac_kwargs': {'lambda_value': 30}}
    kw_args={},
)

selector = SelectFromModel(
    estimator=RandomForestClassifier(
        n_estimators=300,
        max_depth=9,
        random_state=None,
    ),
    threshold="mean",
)

# Define the classifier
net = NeuralNetBinaryClassifier(
    module=AptaNetMLP,
    module__input_dim=None,
    module__hidden_dim=128,
    module__n_hidden=7,
    module__dropout=0.3,
    module__output_dim=1,
    module__use_lazy=True,
    criterion=torch.nn.BCEWithLogitsLoss,
    max_epochs=200,
    lr=0.00014,
    optimizer=torch.optim.RMSprop,
    optimizer__alpha=0.9,
    optimizer__eps=1e-08,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

pipeline = Pipeline(
    [
        ("features", feature_transformer),
        ("selector", selector),
        ("clf", net),
    ]
)

In [38]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6816       0.5000        0.6935  0.0081
      2        0.7010       0.5000        0.6935  0.0090
      3        0.7804       0.5000        0.6935  0.0090
      4        0.7011       0.5000        0.6935  0.0110
      5        0.6776       0.5000        0.6936  0.0110
      6        0.7262       0.5000        0.6936  0.0090
      7        0.6914       0.5000        0.6936  0.0100
      8        0.7281       0.5000        0.6936  0.0100
      9        0.7713       0.5000        0.6936  0.0125
     10        0.7409       0.5000        0.6936  0.0110
     11        0.6732       0.5000        0.6936  0.0150
     12        0.7088       0.5000        0.6936  0.0090
     13        0.8154       0.5000        0.6936  0.0089
     14        0.6870       0.5000        0.6936  0.0090
     15        0.6901       0.5000        0.6936  0.0090
     16        0.7002       0.5

In [39]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.52


### Third workflow
Dataset balancing using under-sampling, while using your own pipeline.

In [40]:
# If you want to build your own aptamer pipeline, you should use the following imports
from sklearn.preprocessing import FunctionTransformer

from pyaptamer.aptanet import AptaNetClassifier, AptaNetPipeline
from pyaptamer.utils._aptanet_utils import pairs_to_features

In [41]:
# OPTIONAL: If you want to use the Neighborhood Cleaning Rule for under-sampling
# NOTE: If you want to use under-sampling, you need to install imbalanced-learn
# and use the Pipeline from imblearn
# %pip install imblearn
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import NeighbourhoodCleaningRule

In [42]:
feature_transformer = FunctionTransformer(
    func=pairs_to_features,
    validate=False,
    # Optional arguments for pairs_to_features
    # example: kw_args={'k': 4, 'pseaac_kwargs': {'lambda_value': 30}}
    kw_args={},
)

# AptaNetClassifier encapsulates the "selector" and "net" mentioned in Workflow 2
clf = AptaNetClassifier()

model = Pipeline(
    [
        ("ncr", NeighbourhoodCleaningRule()),
        ("clf", clf),
    ]
)

pipeline = AptaNetPipeline(estimator=model)

In [43]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

c:\Users\satvm\miniconda3\envs\pyaptamer-latest\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


In [44]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.48


# Implementing `AptaNet` for real-world use cases

## Data preparation
To train the `AptaNetMLP` the notebook uses the dataset used to train the `AptaTrans` algorithm, this dataset can be found in `pyaptamer/datasets/data/train_li2014`.

In [45]:
# Data imports
import numpy as np
import pandas as pd

from pyaptamer.datasets import load_csv_dataset

In [46]:
df = load_csv_dataset("train_li2014")

df = pd.concat([df[:30], df[2300:]])

X_raw, y_raw = df.iloc[:, :2], df.iloc[:, 2]

# Build combinations in the form of (aptamer, protein)
X = list(zip(X_raw.iloc[:, 0], X_raw.iloc[:, 1], strict=False))

# Dummy binary labels for the pairs
y = np.where(y_raw == "positive", 1, 0)

In [47]:
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


## DIfferent Real-world examples
In the second half of this notebook we will explore 3 real-world use cases of the algorithm. These include:
1. Training on the entire AptaTrans dataset, and predict binding probability (`predict_proba`) between a protein and a DNA sequence.
2. same for using DNA sequences from a `fasta` file.
3. Using MCTS combined with a trained `AptaNet` to propose new aptamers for a new pdb file, i.e., a form of in-silico Selex.

In [48]:
# Fit the pipeline on the aptamer-protein pairs from the `AptaTrans` dataset
pipeline.fit(X, y)

C:\Users\satvm\pyaptamer\pyaptamer\pseaac\_pseaac_aptanet.py:196: UserWarning: Invalid amino acid(s) found in sequence. Replaced with 'N'.
  seq = clean_protein_seq(protein_sequence)


### First workflow

Predicting binding probability (`predict_proba`) between a protein (1GNH) and a DNA sequence.

In [49]:
aptamer_sequence = "GGGAGGACGAAGACGACUCGAGACAGGCUAGGGAGGGA"

gnh = load_1gnh_structure()
protein_sequence = struct_to_aaseq(gnh)

X = [(aptamer_sequence, p) for p in protein_sequence]

pipeline.predict_proba(X)

array([[0.53823733, 0.46176267],
       [0.53823733, 0.46176267],
       [0.53823733, 0.46176267],
       [0.53823733, 0.46176267],
       [0.53823733, 0.46176267],
       [0.53823733, 0.46176267],
       [0.53823733, 0.46176267],
       [0.53823733, 0.46176267],
       [0.53823733, 0.46176267],
       [0.53823733, 0.46176267]], dtype=float32)

### Second workflow

Predicting binding probability (`predict_proba`) between a protein (1GNH) and DNA sequences from a `fasta` file.

In [50]:
from pyaptamer.trafos.convert import AnyToAASeq
from pyaptamer.utils import hf_to_dataset

ds = hf_to_dataset(
    "https://huggingface.co/datasets/gcos/HoloRBP4_round8_trimmed/resolve/main/HoloRBP4_round8_trimmed.fasta",
    download_locally=True,
)

transformer = AnyToAASeq(format="fasta")
fasta = transformer.transform("hf_datasets/HoloRBP4_round8_trimmed.fasta")

seqs = fasta["sequence"].tolist()
X = [(aptamer_sequence, seq) for seq in seqs]

pipeline.predict_proba(X[:10])

array([[0.53823286, 0.46176714],
       [0.5382338 , 0.46176618],
       [0.5382333 , 0.46176675],
       [0.53823274, 0.46176726],
       [0.53823316, 0.46176684],
       [0.5382338 , 0.46176618],
       [0.5382329 , 0.46176705],
       [0.5382345 , 0.46176553],
       [0.5382335 , 0.46176648],
       [0.538234  , 0.46176603]], dtype=float32)

### Third workflow

Using MCTS combined with a trained `AptaNet` to propose new aptamers for a protein (1GNH), i.e., a form of in-silico Selex.

In [51]:
from pyaptamer.experiments import AptamerEvalAptaNet
from pyaptamer.mcts import MCTS

gnh = load_1gnh_structure()
protein_sequence = struct_to_aaseq(gnh)

# There can only be one target sequence
eval = AptamerEvalAptaNet(pipeline=pipeline, target=protein_sequence[0])

mcts = MCTS(experiment=eval)

output = mcts.run(verbose=True)


 ----- Round: 1 -----
##################################################
Best subsequence: _G_GG_
Depth: 3
##################################################

 ----- Round: 2 -----
##################################################
Best subsequence: _G_GG__A_GG_
Depth: 6
##################################################

 ----- Round: 3 -----
##################################################
Best subsequence: _G_GG__A_GG__U_A_A
Depth: 9
##################################################

 ----- Round: 4 -----
##################################################
Best subsequence: _G_GG__A_GG__U_A_AC__A_U
Depth: 12
##################################################

 ----- Round: 5 -----
##################################################
Best subsequence: _G_GG__A_GG__U_A_AC__A_U_AG__C
Depth: 15
##################################################

 ----- Round: 6 -----
##################################################
Best subsequence: _G_GG__A_GG__U_A_AC__A_U_AG__CC_G_U_
Depth: 18
####

In [52]:
# Get the best sequence
output["candidate"]

'UGCGCGGGGAGUAAAUACAG'